<h1>The apartment finder:</h1>

<h2>First, we ge the apartment information from our Kijiji scraper:</h2>

In [1]:
                                                                                                                                                                                                    #Get informations from Kijiji scraper:
%run kijiji_scraper.ipynb

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_linux64.zip


[WDM] - Driver has been saved in cache [/home/kurosuzume/.wdm/drivers/chromedriver/linux64/85.0.4183.87]


There are no more apartments with those filters right now. 
chrome closed


In [2]:
import html
from IPython.display import HTML


<h2>The apartment addresses are shown below.</h2>

In [3]:
text_address

['35 Patience Ln, Fredericton, NB E3B 0K4, Canada',
 '504 Priestman St, Fredericton, NB E3B 3B4, Canada',
 '382 George Street, E3B 1J7, Fredericton, NB',
 '382 George Street, Fredericton E3B 1J7 NB',
 '78 York Street, E3B3N5, Fredericton, NB',
 '160 Saint John St., E3B4A9, Fredericton, NB',
 '31 Corbett Ave, E3A 3X3, Fredericton, NB',
 '38 Waterloo Row, e3b3b9, Fredericton, NB',
 '706 Valleyview St, Fredericton, NB E3B 4E8, Canada',
 'Valleyview St, Fredericton, NB E3B 4E8, Canada',
 '1108 Woodstock Rd, Fredericton, NB E3B 7B7, Canada',
 '31 Waggoners Lane, Fredericton, NB, E3B 2L2',
 '15 Waggoners Lane, Fredericton, NB, E3B 2L2',
 '19 Forest Hill Rd, Fredericton, NB, E3B 4J9',
 '527 PRIESTMAN ST, FREDERICTON, NB, E3B 3B6',
 '358 Westmorland St, Fredericton, NB, E3B 3M5',
 '202 Parkside Dr, Fredericton, NB E3B 5V7, Canada',
 'Greenfields Dr, Fredericton, NB E3B 5V9, Canada',
 '990 Onondaga St, Oromocto, NB E2V 1B7, Canada',
 '74 Kilburn St, Fredericton, NB E3A 1Y1, Canada',
 '151 King 

In [4]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h2>Now, we import all libraries needed:</h2>

In [5]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files

!pip install pgeocode
import pgeocode

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't competed 
from geopy.geocoders import Nominatim

#Matplotlib and associated ploting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium-0.5.0 --yes
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from geopy import distance  
print("Libraries imported successfully")



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - folium-0.5.0

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported successfully


<h2>The reference point is the NBCC campus in New Brunswick.</h2>
<h3>We set the location coordinates and the radius we will use to find the apartments and amenities:</h3>

In [6]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000


The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


In [7]:
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(AD_title))

<h3>Ensuring all te data is in the right format:</h3>

In [8]:
#Before finding the geo-locations of the addresses given, we need to verify if they are in the right format:
main_address = []
unaccepted_address_index = []

for address in text_address:
    house = address.split(',')
    house = house[0]+",Fredericton, CA"
    main_address.append(house)

for i in range(len(main_address)):
    if main_address is None:    
        main_address.remove(main_address[i])
        ap_prices.remove(ap_prices[i])
        title_pic_change.remove(title_pic_change[i])
        num_bedrooms.remove(num_bedrooms[i])
        num_bathrooms.remove(num_bathrooms[i])
#print (main_address)
#print(len(main_address))
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
print("Data format verified!")

Data format verified!


In [9]:
geo_location = main_address

<h2>Finding the geographic location of each apartment:</h2>

In [10]:
#Now, to find the geo location of each of the addresses:

index_to_remove = []
geo_location = []
for address in range(len(main_address)):
    appended_location = geolocator.geocode(main_address[address])
    geo_location.append(appended_location)
    if appended_location is None:
        index_to_remove.append(address)
index_to_remove = index_to_remove.reverse()

if index_to_remove is None:
        pass
else: 
    for i in index_to_remove:

        geo_location.remove(geo_location[address])
        ap_prices.remove(ap_prices[address])
        num_bedrooms.remove(num_bedrooms[address])
        num_bathrooms.remove(num_bathrooms[address])          

#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(geo_location))
print("geo location acquired.")

geo location acquired.


<h3>Showing the Location information for all the apartments</h3>

In [11]:
geo_location

[None,
 Location(Priestman Street, Rabbit Town, Uptown, Fredericton, York County, New Brunswick, E3B 3H2, Canada, (45.9445916, -66.6649631, 0.0)),
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(York Street, Rabbit Town, Downtown, Fredericton, York County, New Brunswick, E3B 0V4, Canada, (45.9546808, -66.6503261, 0.0)),
 Location(Saint John Street, Queen's Park, Downtown, Fredericton, York County, New Brunswick, E3C 5H1, Canada, (45.9554708, -66.6405396, 0.0)),
 Location(31, Corbett Avenue, Station Pointe, Barker's Point, Fredericton, York County, New Brunswick, E3A 6R1, Canada, (45.95220965, -66.62178275, 0.0)),
 Location(36-38, Waterloo Row, Town Platt, Downtown, Fredericton, York County, New Brunswick, E3B 4C9, Canada, (45.9551557, -66.63313950579706, 0

In [12]:
location_info = []
discarded_index = []
for location in range(len(geo_location)):
    if not geo_location[location] == None: 
        location_info.append([geo_location[location].address, geo_location[location].latitude, geo_location[location].longitude ])
    else: 
        discarded_index.append(location)
    

In [13]:
#print(location_info)
#print(len(location_info))

<h3>Using the Introduction_to_the_Problem notebook to retrieve the map.</h3>

In [14]:
#Get the map code from the introduction notebook:
%run Introduction_to_the_Problem.ipynb

The college address is: Duffie Drive, College Hill, Skyline Acres, Fredericton, York County, New Brunswick, E3B 9W4, Canada


In [15]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000



The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Foursquare credentials:</h3>
<p>Please, fill with your foursquare information:</p>

In [16]:
CLIENT_ID = 'GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4' # your Foursquare ID
CLIENT_SECRET = 'LTMU32GFPWTBGXJ0HGGDK2VNAFU3SXQ1FZMGJEX3XBTBULNK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID + 'PLEASE VERIFY!')
print('CLIENT_SECRET:' + CLIENT_SECRET + 'PLEASE VERIFY!')

Your credentails:
CLIENT_ID: GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4PLEASE VERIFY!
CLIENT_SECRET:LTMU32GFPWTBGXJ0HGGDK2VNAFU3SXQ1FZMGJEX3XBTBULNKPLEASE VERIFY!


<p>Since it wasn't possible to use an automatic iterative method to find the amenities, we will do it manually. for that we will use the Foursquare API and acquire nearby venues data:</p>

In [17]:
#Now, to get the nearby venues according to the important amenities:
amenities_list = ["Grocery Store","Supermarket", "Gym", "Drugstore", "Hospital", "Candy Store"]
search_query0 = "Grocery Store"
search_query1 = "Supermarket"
search_query2 = "Gym"
search_query3 = "Drugstore"
search_query4 = "Hospital"
search_query5 = "Candy Store"


In [18]:
url0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query0, radius, LIMIT)
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query4, radius, LIMIT)
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query5, radius, LIMIT)


In [19]:
results0 = requests.get(url0).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()
results5 = requests.get(url5).json()



In [20]:
# assign relevant part of JSON to venues
venues0 = results0['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']
venues5 = results5['response']['venues']

# transform venues into dataframes
dataframe0 = json_normalize(venues0)
dataframe1 = json_normalize(venues1)
dataframe2 = json_normalize(venues2)
dataframe3 = json_normalize(venues3)
dataframe4 = json_normalize(venues4)
dataframe5 = json_normalize(venues5)


/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use p

In [21]:
print("Venues Data retrieved.")

Venues Data retrieved.


<h2>Results from foursquare will converge into one dataframe:</h2>

In [22]:
# now converge all results into one dataframe:

dataframe0 = pd.concat([dataframe0,dataframe1], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe2], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe3], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe4], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe5], axis=0, ignore_index=True)



dataframe0

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c545e5db426ef3b11cc7e8a,Dick's Grocery Store,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",v-1601145798,False,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,NaN
1,4c3c8e25282203bb55a3fcda,American Eagle & Aerie Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1601145798,False,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,NaN
2,4badfab8f964a520a3773be3,Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1601145798,False,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
3,53f7c437498eacb60d222490,Great Canadian Dollar Store,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",v-1601145798,False,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,NaN
4,505ba09be4b027df9b199de0,Jump +,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",v-1601145798,False,1381 Regent St,45.935711,-66.664615,"[{'label': 'display', 'lat': 45.93571105673426...",1673,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent St, Fredericton NB E3C 1A2, Canada]",NaN,NaN
5,4ba6479cf964a520de4139e3,Urban Almanac,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",v-1601145798,False,NaN,45.962546,-66.643497,"[{'label': 'display', 'lat': 45.96254611483269...",2195,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
6,4dbf1b1b6e810768bf9ccf5a,Raymonde's General Store,[],v-1601145798,False,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
7,50a8f005e4b0e4f42e033a2a,Gap Factory Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1601145798,False,"11 Trinity Ave, Fredericton",45.930211,-66.662416,"[{'label': 'display', 'lat': 45.930211, 'lng':...",1918,E3C 0B9,CA,Fredericton,NB,Canada,"[11 Trinity Ave, Fredericton, Fredericton NB E...",NaN,NaN
8,4c1030d7ce57c928688981d2,UPS Store,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",v-1601145798,False,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,NaN
9,4be89132d837c9b6447fa506,University Cafe & Market Convenience Store,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",v-1601145798,False,University Avenue,45.953861,-66.635277,"[{'label': 'display', 'lat': 45.95386136941374...",1460,NaN,CA,Fredericton,NB,Canada,"[University Avenue (Shore St), Fredericton NB,...",Shore St,NaN


In [23]:
dataframe0.shape

(94, 18)

<h2>Filtering the pandas dataframe:</h2>

In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe0.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe0.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Dick's Grocery Store,Smoke Shop,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,4c545e5db426ef3b11cc7e8a
1,American Eagle & Aerie Store,Clothing Store,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,4c3c8e25282203bb55a3fcda
2,Dollar Store,Miscellaneous Shop,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4badfab8f964a520a3773be3
3,Great Canadian Dollar Store,Discount Store,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,53f7c437498eacb60d222490
4,Jump +,Electronics Store,1381 Regent St,45.935711,-66.664615,"[{'label': 'display', 'lat': 45.93571105673426...",1673,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent St, Fredericton NB E3C 1A2, Canada]",NaN,505ba09be4b027df9b199de0
5,Urban Almanac,Furniture / Home Store,NaN,45.962546,-66.643497,"[{'label': 'display', 'lat': 45.96254611483269...",2195,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4ba6479cf964a520de4139e3
6,Raymonde's General Store,None,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4dbf1b1b6e810768bf9ccf5a
7,Gap Factory Store,Clothing Store,"11 Trinity Ave, Fredericton",45.930211,-66.662416,"[{'label': 'display', 'lat': 45.930211, 'lng':...",1918,E3C 0B9,CA,Fredericton,NB,Canada,"[11 Trinity Ave, Fredericton, Fredericton NB E...",NaN,50a8f005e4b0e4f42e033a2a
8,UPS Store,Paper / Office Supplies Store,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,4c1030d7ce57c928688981d2
9,University Cafe & Market Convenience Store,Candy Store,University Avenue,45.953861,-66.635277,"[{'label': 'display', 'lat': 45.95386136941374...",1460,NaN,CA,Fredericton,NB,Canada,"[University Avenue (Shore St), Fredericton NB,...",Shore St,4be89132d837c9b6447fa506


In [25]:
amenities= dataframe_filtered["name"].to_list()
amenities_categories = dataframe_filtered["categories"].to_list()
amenities_list_lat = dataframe_filtered["lat"].to_list()
amenities_list_lng = dataframe_filtered["lng"].to_list()


<h3>And finally, we add the amenities to our Folium map.</h3>
<p>The amenities will be displayed as a blue dot with brown outline. </p>

In [26]:

for i in range(len(geo_location)):
    if not geo_location[i] == None: 
        tooltip = "Apartment"
        folium.Marker([geo_location[i].latitude,geo_location[i].longitude], popup=AD_title[i], tooltip = tooltip ).add_to(map_Fred)
    else:
        pass

In [27]:

# add the Italian amenities as blue circle markers with red outline
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip = label
    ).add_to(map_Fred)
map_Fred

<h3>Now, using geopy, we can verify amenities within a 800m radius from an apartment.</h3>

In [28]:
from geopy import point
#Function to find amenities in a 500m radius from a center point:
def find_in_radius(center_name, center_lat,center_lng, amenities_list_lat, amenities_list_lng, amenities_list, amenities_categories):
    """(float, float, list) -> list
        Retruns a list of amenities within a 800 radius from a center point.
    """
    center_lat = str(center_lat)
    center_lng = str(center_lng)
    center_point = point.Point(str(center_lat)+ "," + str(center_lng))
    
    within_radius = []
    amenities_cat = []
    for amenity in range(len(amenities_list)):
        amenities_list_point = point.Point(str(amenities_list_lat[amenity])+ "," + str(amenities_list_lng[amenity]))
        amen_dis = distance.distance(center_point, amenities_list_point).km
        if amen_dis <= 0.80: #1000 meters
            within_radius.append(amenities_list[amenity])
            amenities_cat.append(amenities_categories[amenity])
    return center_name,within_radius,amenities_cat

        

In [29]:
#num_bedrooms
#num_bathrooms
#appartment_price_accepted
for type in range(len(amenities_categories)):
    if amenities_categories[type] == None:
        amenities_categories[type] = "None"

In [30]:
full_list = []
for i in range(len(location_info)):
    full_list.append(find_in_radius(location_info[i][0],location_info[i][1], location_info[i][2], amenities_list_lat, amenities_list_lng, amenities,amenities_categories))

In [31]:
#print(discarded_index)
#print(len(ap_prices))


In [32]:
reverse_discarded_index = discarded_index.reverse()

In [33]:
#discarded_index


In [34]:
#len(ap_prices)

In [35]:
for i in discarded_index:
    ap_prices.remove(ap_prices[i])
    num_bedrooms.remove(num_bedrooms[i])
    num_bathrooms.remove(num_bathrooms[i])

In [36]:
#len(ap_prices)
#len(num_bedrooms)

In [37]:
ap_prices_float = []
for i in range(len(ap_prices)):
    split = ap_prices[i].replace('$', '')
    
    ap_prices_float.append(split)
    

In [38]:
ap_prices_final = []
for i in range(len(ap_prices_float)):
    make_float = float(ap_prices_float[i].replace(',',''))
    ap_prices_final.append(make_float)
ap_prices = ap_prices_final

In [39]:
apt_address = []
amenity_name = []
amenity_type= []
for i in range(len(full_list)):
    apt_address.append(full_list[i][0])
    amenity_name.append(full_list[i][1])
    amenity_type.append(full_list[i][2])

In [40]:
separator = ","
prepared_amenity = []
for i in range(len(amenity_type)):
    sep = separator.join(amenity_type[i])
    prepared_amenity.append(sep)


In [41]:
for i in range(len(prepared_amenity)):
    prepared_amenity[i] = str(prepared_amenity[i]).split(',')
    for j in range(len(prepared_amenity[i])):
        prepared_amenity[i][j].split(",")

    

In [50]:
prepared_amenity

[['Smoke Shop',
  'Discount Store',
  'Department Store',
  'Miscellaneous Shop',
  'Veterinarian',
  'Department Store',
  'Miscellaneous Shop',
  'Discount Store',
  'Smoke Shop'],
 ['Furniture / Home Store',
  'Paper / Office Supplies Store',
  'Miscellaneous Shop',
  'Convenience Store',
  'General Travel',
  'Clothing Store',
  'Tourist Information Center',
  'Gym / Fitness Center',
  'Veterinarian',
  'Nudist Beach',
  'Furniture / Home Store',
  'Paper / Office Supplies Store',
  'Miscellaneous Shop',
  'Convenience Store',
  'General Travel',
  'Clothing Store',
  'Tourist Information Center'],
 ['Furniture / Home Store',
  'Paper / Office Supplies Store',
  'Miscellaneous Shop',
  'Convenience Store',
  'General Travel',
  'Clothing Store',
  'Tourist Information Center',
  'Gym / Fitness Center',
  'Veterinarian',
  'Nudist Beach',
  'Furniture / Home Store',
  'Paper / Office Supplies Store',
  'Miscellaneous Shop',
  'Convenience Store',
  'General Travel',
  'Clothing Stor

In [42]:
apartment_df= pd.DataFrame(list(zip(apt_address, amenity_name, prepared_amenity, ap_prices, num_bedrooms, num_bathrooms)))

In [43]:
len(prepared_amenity)
result = list(set(x for l in prepared_amenity for x in l))


In [51]:
result

['',
 'Gym / Fitness Center',
 'Furniture / Home Store',
 'Nudist Beach',
 'Clothing Store',
 'College Gym',
 'Veterinarian',
 'Department Store',
 'Miscellaneous Shop',
 'None',
 'General Travel',
 'Smoke Shop',
 'Discount Store',
 'Convenience Store',
 'College Rec Center',
 'Paper / Office Supplies Store',
 'Hospital',
 'Candy Store',
 'Tourist Information Center']

In [44]:
apartment_df = pd.concat([apartment_df,pd.get_dummies(apartment_df[2].apply(pd.Series).stack()).sum(level=0)], axis = 1)

In [45]:
apartment_df = apartment_df.replace({5:{'Bathrooms: 1': 1, 'Bathrooms: 2': 2, "Bathrooms: 1.5": 2}})

apartment_df = apartment_df.replace({4:{'Bedrooms: Bachelor/Studio': 0, 'Bedrooms: 1': 1,'Bedrooms: 2': 2, 'Bedrooms: 1 + Den':2}})


In [46]:
apartment_df = apartment_df.rename(columns={0: "Apartment Address", 1: "Amenities",2:"Amenity type", 3:"Rent price", 4:"Bedrooms", 5:"Bathrooms"})


In [47]:
#apartment_df["Bedrooms"].astype("int32").dtypes

In [48]:
#apartment_df["Bathrooms"].astype("int32").dtypes

<h3>with all that information, we're able to create the apartment DataFrame:</h3>

In [49]:
apartment_df.head()

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,,Candy Store,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym / Fitness Center,Hospital,Miscellaneous Shop,None,Nudist Beach,Paper / Office Supplies Store,Smoke Shop,Tourist Information Center,Veterinarian
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[Dick's Grocery Store, Great Canadian Dollar S...","[Smoke Shop, Discount Store, Department Store,...",450.0,0,1,0,0,0,0,0,0,2,2,0,0,0,0,2,0,0,0,2,0,1
1,"George Street, Carleton, Downtown, Fredericton...","[Urban Almanac, UPS Store, The UPS Store, Kwik...","[Furniture / Home Store, Paper / Office Suppli...",500.0,0,1,0,0,2,0,0,2,0,0,2,2,1,0,2,0,1,2,0,2,1
2,"George Street, Carleton, Downtown, Fredericton...","[Urban Almanac, UPS Store, The UPS Store, Kwik...","[Furniture / Home Store, Paper / Office Suppli...",550.0,0,1,0,0,2,0,0,2,0,0,2,2,1,0,2,0,1,2,0,2,1
3,"York Street, Rabbit Town, Downtown, Fredericto...","[UPS Store, The UPS Store, Kwik-Way Convenienc...","[Paper / Office Supplies Store, Miscellaneous ...",600.0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,2,0,0,1
4,"Saint John Street, Queen's Park, Downtown, Fre...","[UPS Store, University Cafe & Market Convenien...","[Paper / Office Supplies Store, Candy Store, M...",600.0,0,1,0,2,2,0,0,0,0,0,0,2,1,0,2,0,1,2,0,0,1
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[],700.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","[University Cafe & Market Convenience Store, T...","[Candy Store, General Travel, Clothing Store, ...",748.0,0,1,0,2,2,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0
7,"706, Valleyview Street, College Hill, Skyline ...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[Miscellaneous Shop, Gym / Fitness Center, Col...",750.0,1,1,0,0,0,2,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0
8,"Valleyview Street, College Hill, Skyline Acres...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[Miscellaneous Shop, Gym / Fitness Center, Col...",750.0,1,1,0,0,0,2,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[],775.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h3>Then, we open the csv file with an older instance of the DF, and append the unique new values between them</h3>

In [ ]:
apartment_df_initial = pd.read_csv("apartment_dataframe.csv")


In [ ]:
apartment_df = pandas.concat([apartment_df_initial,apartment_df]).drop_duplicates().reset_index(drop=True)

In [52]:
apartment_df.to_csv("apartment_dataframe.csv",mode='a', header=False, index=True)

print("The apartment_dataframe.csv has been updated!")

<p>Now, we will use this information to analyse the results and generate a conclusion.</p>